In [40]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import pandas as pd
import os
import dotenv
import cloudinary
import cloudinary.uploader
from cloudinary.utils import cloudinary_url

dotenv.load_dotenv()

True

In [41]:
cloudinary.config(
    cloud_name=os.getenv("CLOUDINARY_CLOUD_NAME"),
    api_key=os.getenv("CLOUDINARY_API_KEY"),
    api_secret=os.getenv("CLOUDINARY_API_SECRET"),
    secure=True
)

# Firebase Init

In [42]:
service_account_info = {
    "type": os.getenv("FIREBASE_TYPE"),
    "project_id": os.getenv("FIREBASE_PROJECT_ID"),
    "private_key_id": os.getenv("FIREBASE_PRIVATE_KEY_ID"),
    "private_key": os.getenv("FIREBASE_PRIVATE_KEY"),
    "client_email": os.getenv("FIREBASE_CLIENT_EMAIL"),
    "client_id": os.getenv("FIREBASE_CLIENT_ID"),
    "auth_uri": os.getenv("FIREBASE_AUTH_URI"),
    "token_uri": os.getenv("FIREBASE_TOKEN_URI"),
    "auth_provider_x509_cert_url": os.getenv("FIREBASE_AUTH_PROVIDER_X509_CERT_URL"),
    "client_x509_cert_url": os.getenv("FIREBASE_CLIENT_CERT_URL"),
    "universe_domain": os.getenv("FIREBASE_UNIVERSE_DOMAIN")
}

In [36]:
if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_info)
    firebase_admin.initialize_app(cred, {
        'storageBucket': 'coffee-shop-app-a4152.appspot.com',
        'databaseURL': 'https://coffeeshop-app-49b49-default-rtdb.firebaseio.com/'
    })
else:
    print("Firebase app already initialized")

Firebase app already initialized


# Upload Data

In [43]:
image_folder_path = './products/images/' # Path to the folder containing images

In [44]:
products_collection = db.reference('products')

In [45]:
df = pd.read_json('products/products.jsonl', lines=True)

In [46]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [47]:
def upload_image(image_path):  # ⚠️ Removed 'bucket' parameter
    """Upload image to Cloudinary and return public URL"""
    image_name = image_path.split('/')[-1]
    
    # Upload to Cloudinary with folder organization
    response = cloudinary.uploader.upload(
        image_path,
        folder="coffee-shop/product_images",  # Organize in folders
        public_id=image_name.split('.')[0],   # Remove file extension
        overwrite=True,
        resource_type="image"
    )
    
    # Return the secure public URL (like Firebase's public_url)
    return response['secure_url']


In [48]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [50]:
for index, row in df.iterrows():
    print(index, row['name'])
    image_path = os.path.join(image_folder_path, row['image_path'])
    
    # MODIFIED: No longer passing 'bucket' parameter
    image_url = upload_image(image_path)  # ⚠️ Changed function call
    
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url'] = image_url
    
    # Save to Firebase Realtime Database (unchanged)
    products_collection.push().set(product_data)

print("✅ All products uploaded successfully!")

0 Cappuccino
1 Jumbo Savory Scone
2 Latte
3 Chocolate Chip Biscotti
4 Espresso shot
5 Hazelnut Biscotti
6 Chocolate Croissant
7 Dark chocolate
8 Cranberry Scone
9 Croissant
10 Almond Croissant
11 Ginger Biscotti
12 Oatmeal Scone
13 Ginger Scone
14 Chocolate syrup
15 Hazelnut syrup
16 Carmel syrup
17 Sugar Free Vanilla syrup
✅ All products uploaded successfully!
